# Loading the data into a dataframe from wikipedia page - Part 1

In [1]:
import pandas as pd
import numpy as np
#!conda install -c conda-forge beautifulsoup4 --yes # installing bs4 package # uncomment for the 1st time to install the package
#!conda install -c conda-forge html5lib --yes # installing html5lib package alrady available in envt
#!conda install -c conda-forge lxml --yes # lxml’s HTML parser can retrieve html pages
#!conda install -c conda-forge geocoder --yes
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if it is loaded once
import folium # map rendering library
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import requests
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

creating a Beautiful soup object from the url : 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' using lxml parser

In [2]:
soup = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text,'lxml')    
#print(soup.prettify()) # to view the data in a proper way

Extracting table data from the wikipedia using the class : 'wikitable sortable'

In [3]:
Table = soup.find('table',{'class':'wikitable sortable'})


In [4]:
#Table

Extracting the data from the table content and loading data to a dataframe which will useful for further analysis

In [5]:
#df = pd.read_table(Table)
#for df in pd.read_html()
#df = pd.DataFrame(columns=['Postcode','Borough','Neighbourhood'])
#print(df)
#df['Postcode'] =trs[0].text
#df['Borough']=trs[1].a['title']
#df['Neighbourhood'] = trs[2].a['title']
#df['Postcode'] =trs[0].text
#df['Borough']=trs[1].text
#df['Neighbourhood'] = trs[2].text
records = []
for tr in Table.findAll("tr"):
    trs = tr.findAll("td")
    if(trs!=[]):
        record=[]
        record.append(trs[0].text)
        record.append(trs[1].text)
        record.append(trs[2].text)
        records.append(record)
               
    
df = pd.DataFrame(data=records,columns = ['Postal Code','Borough','Neighbourhood'])


In [6]:
df['Neighbourhood']= df['Neighbourhood'].str.rstrip('\n')

Dropping the rows where the Borough data is `Not assigned`

In [7]:
df.drop(df[df['Borough'] =='Not assigned'].index,axis=0,inplace=True)

In [8]:
df['Neighbourhood']=df['Neighbourhood'].mask(df['Neighbourhood']=='Not assigned',df['Borough'])


In [9]:
df.reset_index(drop=True)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


Reference for merging the data https://stackoverflow.com/questions/52785911/how-to-combine-rows-on-common-categorical-column-in-pandas

In [10]:
df = df.pivot_table(index = ['Postal Code','Borough'],values = 'Neighbourhood',aggfunc=','.join)

In [11]:
df

,,Neighbourhood
Postal Code,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"


In [12]:
df.reset_index(inplace=True)

In [13]:
df.shape

(103, 3)

# Loading latitude longitude from CSV file - Part 2

In [14]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_2a57edb43d434b9489ee5c83cf1c99a6 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='aGzgi0LASQnEuUKUutOhfPt1Mh86KNxBq0A48apWiYJG',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_2a57edb43d434b9489ee5c83cf1c99a6.get_object(Bucket='datascience-donotdelete-pr-ij1soqkjyiglce',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df = pd.merge(df, df_data_1, on='Postal Code')

In [16]:
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


# Explore and clustering the neighborhoods in Toronto - part 3

In [17]:
print("Data Frame contains {}  borough and {} neighbourhoods. ".format(len(df['Borough'].unique()),df.shape[0]))

Data Frame contains 11  borough and 103 neighbourhoods. 


 Using Geopy library for getting latitute and longitude values

In [18]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent= "ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('geo coordinates of Toronto canada are {} {} '.format(latitude,longitude))


geo coordinates of Toronto canada are 43.653963 -79.387207 


Creating a map of Toronto with neighborhoods

In [19]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [20]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [21]:
toronto_data[-1:]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
38,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242


Four Square API creds

In [22]:
CLIENT_ID = 'CXST52NKZIR04JODAXOQX1P3M0U43KBWUEDQLNMQ2B4RC2TL' # your Foursquare ID
CLIENT_SECRET = '42DG3PTXLR242ITY13S0XLMLSMFGKZZWQWVX25F5UCES4CMY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CXST52NKZIR04JODAXOQX1P3M0U43KBWUEDQLNMQ2B4RC2TL
CLIENT_SECRET:42DG3PTXLR242ITY13S0XLMLSMFGKZZWQWVX25F5UCES4CMY


In [23]:
LIMIT = 300 # for getting number of results

Creating a function to explore all the neighbourhoods in Toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
test_df = toronto_data[toronto_data['Neighbourhood'] == "The Beaches"]
toronto_queens_park = getNearbyVenues(names=test_df['Neighbourhood'],
                                   latitudes=test_df['Latitude'],
                                   longitudes=test_df['Longitude']
                                  )

The Beaches


In [26]:
toronto_queens_park

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [27]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [28]:
toronto_venues[toronto_venues['Neighbourhood']=="Queen's Park"]




,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [29]:
toronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton,Exhibition Place,Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
"Chinatown,Grange Park,Kensington Market",94,94,94,94,94,94
Christie,17,17,17,17,17,17


Unique categories of venues

In [30]:
print('{} unique categories. '.format(len(toronto_venues['Venue Category'].unique())))

232 unique categories. 


# Analysing Neighbourhood

In [31]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix="",prefix_sep="") # one hot encoding

# adding neighbourhood column to onehot encoded toronto df

toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']

# making neighbourhood column to 1st column
fixed_cols = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_cols]


In [32]:
toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
toronto_onehot.shape

(1685, 233)

In [34]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,...,0.000000,0.000000,0.00000,0.00,0.011905,0.000000,0.000000,0.011905,0.000000,0.011905
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010638,0.000000,0.00000,0.00,0.031915,0.000000,0.042553,0.010638,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.012048,0.000000,0.012048,0.012048


In [35]:
toronto_grouped.shape

(38, 233)

Printing top 10 common venues of each neighbourhoood

In [36]:
num_top_venues = 10

for hood in toronto_grouped['Neighbourhood']:
    print("----"+ hood +" -------")
    temp = toronto_grouped[toronto_grouped['Neighbourhood']==hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending = False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond -------
              venue  freq
0       Coffee Shop  0.07
1              Café  0.05
2   Thai Restaurant  0.04
3        Steakhouse  0.04
4        Restaurant  0.03
5       Salad Place  0.03
6      Burger Joint  0.03
7  Sushi Restaurant  0.03
8  Asian Restaurant  0.03
9               Bar  0.03


----Berczy Park -------
                 venue  freq
0          Coffee Shop  0.09
1         Cocktail Bar  0.04
2           Steakhouse  0.04
3   Seafood Restaurant  0.04
4          Cheese Shop  0.04
5       Farmers Market  0.04
6               Bakery  0.04
7                 Café  0.04
8             Beer Bar  0.04
9  Sporting Goods Shop  0.02


----Brockton,Exhibition Place,Parkdale Village -------
                    venue  freq
0          Breakfast Spot  0.09
1                    Café  0.09
2             Coffee Shop  0.09
3                     Gym  0.05
4                  Bakery  0.05
5           Burrito Place  0.05
6            Climbing Gym  0.05
7       Convenience S

                      venue  freq
0                Playground  0.33
1              Intersection  0.33
2                Restaurant  0.33
3         Afghan Restaurant  0.00
4             Movie Theater  0.00
5        Mac & Cheese Joint  0.00
6                    Market  0.00
7            Massage Studio  0.00
8  Mediterranean Restaurant  0.00
9               Men's Store  0.00


----North Toronto West -------
                  venue  freq
0        Clothing Store  0.14
1   Sporting Goods Shop  0.09
2           Coffee Shop  0.09
3           Yoga Studio  0.05
4  Gym / Fitness Center  0.05
5    Mexican Restaurant  0.05
6                 Diner  0.05
7          Dessert Shop  0.05
8                  Park  0.05
9        Cosmetics Shop  0.05


----Parkdale,Roncesvalles -------
                         venue  freq
0                    Gift Shop  0.14
1                      Dog Run  0.07
2                    Bookstore  0.07
3                Movie Theater  0.07
4  Eastern European Restaurant  0.07
5    

In [37]:
#temp = toronto_grouped[toronto_grouped['Neighbourhood']==hood].T.reset_index()
#temp.iloc[1:]

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Salad Place,Burger Joint,Bar,Bakery,Sushi Restaurant,Asian Restaurant
1,Berczy Park,Coffee Shop,Steakhouse,Cocktail Bar,Café,Cheese Shop,Beer Bar,Bakery,Seafood Restaurant,Farmers Market,Bistro
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Gym,Restaurant,Pet Store,Performing Arts Venue,Italian Restaurant,Intersection
3,Business Reply Mail Processing Centre 969 Eastern,Burrito Place,Smoke Shop,Fast Food Restaurant,Farmers Market,Spa,Pizza Place,Light Rail Station,Recording Studio,Garden,Garden Center
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Rental Car Location,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina


In [46]:
neighbourhoods_venues_sorted.shape

(38, 11)

# Clustering Neighbourhoods

In [47]:
kclusters = 5 # setting num of clusters
toronto_group_clustering = toronto_grouped.drop('Neighbourhood',1)

kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(toronto_group_clustering)
kmeans.labels_[0:20]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
      dtype=int32)

In [48]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data[:-1]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged  # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Other Great Outdoors,Trail,Pub,Neighborhood,Health Food Store,Doner Restaurant,Dog Run,Donut Shop,Dance Studio,Discount Store
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Bookstore,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Sandwich Place,Park,Board Shop,Ice Cream Shop,Fish & Chips Shop,Pub,Sushi Restaurant,Fast Food Restaurant,Food & Drink Shop,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Fish Market,Ice Cream Shop,Seafood Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Gym / Fitness Center,Swim School,Bus Line,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Hotel,Park,Clothing Store,Sandwich Place,Pizza Place,Food & Drink Shop,Breakfast Spot,Gym,Donut Shop,Doner Restaurant
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Gym / Fitness Center,Health & Beauty Service,Diner,Dessert Shop,Mexican Restaurant,Cosmetics Shop
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Gym,Café,Sushi Restaurant,Pharmacy,Costume Shop
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,4,Restaurant,Playground,Intersection,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,0,Pub,Coffee Shop,American Restaurant,Supermarket,Sushi Restaurant,Sports Bar,Pizza Place,Fried Chicken Joint,Bagel Shop,Restaurant


 visualising the clusters

In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Exploring the clusters


### Cluster 1

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels']==0,toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Other Great Outdoors,Trail,Pub,Neighborhood,Health Food Store,Doner Restaurant,Dog Run,Donut Shop,Dance Studio,Discount Store
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Bookstore,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place
2,East Toronto,0,Sandwich Place,Park,Board Shop,Ice Cream Shop,Fish & Chips Shop,Pub,Sushi Restaurant,Fast Food Restaurant,Food & Drink Shop,Italian Restaurant
3,East Toronto,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Fish Market,Ice Cream Shop,Seafood Restaurant,Sandwich Place
5,Central Toronto,0,Hotel,Park,Clothing Store,Sandwich Place,Pizza Place,Food & Drink Shop,Breakfast Spot,Gym,Donut Shop,Doner Restaurant
6,Central Toronto,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Gym / Fitness Center,Health & Beauty Service,Diner,Dessert Shop,Mexican Restaurant,Cosmetics Shop
7,Central Toronto,0,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Gym,Café,Sushi Restaurant,Pharmacy,Costume Shop
9,Central Toronto,0,Pub,Coffee Shop,American Restaurant,Supermarket,Sushi Restaurant,Sports Bar,Pizza Place,Fried Chicken Joint,Bagel Shop,Restaurant
11,Downtown Toronto,0,Coffee Shop,Restaurant,Pub,Italian Restaurant,Bakery,Pizza Place,Flower Shop,Café,Grocery Store,Breakfast Spot
12,Downtown Toronto,0,Sushi Restaurant,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Gastropub,Café,Hotel,Mediterranean Restaurant,Gym


## Cluster 2

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels']==1,toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,1,Park,Playground,Trail,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
23,Central Toronto,1,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


## Cluster 3

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels']==2,toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Park,Gym / Fitness Center,Swim School,Bus Line,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


## Cluster 4

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels']==3,toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,3,Health & Beauty Service,Garden,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


## Cluster 5

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels']==4,toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,4,Restaurant,Playground,Intersection,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
